In [1]:
import os
import pandas as pd

# 读取所有数据

In [2]:
def read_excel_files(directory, num_skiprows):
    all_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx") or filename.endswith(".xls"):
            file_path = os.path.join(directory, filename)
            df = pd.read_excel(file_path, skiprows=num_skiprows)
            all_data.append(df)
    return all_data


# 指定目录路径
directory_path = "../data/IoTDamper_v5/第五版物联网风阀320×250/"
# 调用函数读取目录下的所有Excel文件
dfs = read_excel_files(directory_path, num_skiprows=4)

In [3]:
# 查看读取的一个 df
df = dfs[0]
df.head()

,前阀片组开度,后阀片开度,风量罩风量（m3/h）,风阀+连接件前后压差（Pa）,风阀读取压差（Pa）
0,90,90,131,0.0,1.5
1,90,75,130,0.0,1.5
2,90,60,129,0.0,1.6
3,90,45,129,0.0,1.6
4,90,30,127,0.0,2.4


# 划分数据集

In [4]:
val_indexs = [9, 15, 21, 23, 30, 32, 37, 44]
val_indexs = [ids - 6 for ids in val_indexs]  # 每个 df 中作为验证数据的索引
# val_index = df.index.isin(val_indexs)
val_indexs

[3, 9, 15, 17, 24, 26, 31, 38]

In [5]:
df_val = pd.DataFrame()
df_train = pd.DataFrame()

for df in dfs:
    val_rows = df[df.index.isin(val_indexs)]   # 提取索引行
    train_rows = df[~df.index.isin(val_indexs)]   # 提取剩余行

    df_val = pd.concat([df_val, val_rows])   # 将索引行添加到 df_val
    df_train = pd.concat([df_train, train_rows])   # 将剩余行添加到 df_train


In [6]:
df_val

,前阀片组开度,后阀片开度,风量罩风量（m3/h）,风阀+连接件前后压差（Pa）,风阀读取压差（Pa）
3,90,45,129,0.0,1.6
9,75,60,130,0.0,1.6
15,60,75,130,0.0,1.5
17,60,45,129,0.0,1.6
24,45,45,125,3.6,1.6
...,...,...,...,...,...
17,60,45,1796,254.9,290.0
24,45,45,1670,308.7,337.6
26,45,15,1131,474.7,557.5
31,30,45,1390,409.0,425.4


# 储存数据

In [7]:
# 创建 ExcelWriter 对象，并指定文件名
writer = pd.ExcelWriter('../data/integrated.xlsx', engine='xlsxwriter')

# 将 df_train 存储到 "train" 工作表中
df_train.to_excel(writer, sheet_name='train', index=False)
# 将 df_val 存储到 "val" 工作表中
df_val.to_excel(writer, sheet_name='val', index=False)

# 保存并关闭 ExcelWriter
writer.close()